### Calculate double integration filters for Datawell MkIII and Mk4 Waverider buoys

In [ ]:
## Refer to Double Integration and Band Pass Filter document supplied by Datawell

using Printf
using DSP, Plots
using SpecialFunctions
using QuadGK

# Define the structure of buoy filter parameters
struct Buoy    
    fs::Float64
    flo::Float64
    fup::Float64
    M::Int
    alpha::Float64
    P0::Int
    color1::String
    color2::String
    label::String
end
   
# Add buoy filter parameters to structure
DWR4 = Buoy(2.56, 0.03333, 1.0, 255, 9.3, 123, "darkred", "yellow", "DWR4")
MkIII = Buoy(3.84, 0.034, 0.61, 511, 8, 178, "palegreen", "darkgreen", "MkIII")

buoy_types = [MkIII, DWR4]

# Initialize the plot
p = plot(size = (900, 600), xlims=(-150, 150), ylims=(-0.6, 0.2), 
        title="Datawell Double Integration filter coefficients", xlabel="Time (s)", ylabel="Filter coefficients (s²)",
        leftmargin=15Plots.mm, bottommargin=15Plots.mm,
        fg_legend=:false, bg_legend=:transparent, framestyle=:box)

@time begin
# Process both DWR4 and MkIII buoys
for buoy in buoy_types
    
    # Calculate f0
    f0 = 1 / buoy.P0
    
    # Create vector of time values over length of filter
    time = -buoy.M:buoy.M

    # Preallocate filter coefficient array
    dt = zeros(length(time))

    # Vectorized calculation of filter coefficients
    # Handle t = 0 separately for direct calculation using Eq. 4
    dt[time .== 0] .= -1 / (2 * pi^2) * (1 / buoy.flo - 1 / buoy.fup)

    # For other values of t, calculate using quadgk
    non_zero_indices = time .!= 0
    t_non_zero = time[non_zero_indices]

    # Define the function for integration using Eq. 7
    function df(x, t)
        cos(2 * pi * x * t) ./ (x .^ 2 .+ f0^2)    
    end

    # Perform integration using vectorized operations
    dt[non_zero_indices] .= -1 / (2 * pi^2) * [quadgk(x -> df(x, t), buoy.flo, buoy.fup)[1] for t in t_non_zero]

    # Generate a Kaiser window over the filter range using alpha value
    window_function = kaiser(2 * buoy.M + 1, buoy.alpha)

    # Apply the window function and normalize
    normalized_dt = dt .* window_function / buoy.fs

    # Plot the filter
    plot!(time, normalized_dt, c = buoy.color1, lw = 2, label = buoy.label*"\n")

end

display(p)
end

### Calculate Datawell DWR-G filter coefficients

In [ ]:
using Printf
using DSP, Plots
using SpecialFunctions
using QuadGK

# Parameters as individual variables
fₛ = 2.0        # Sampling frequency
fₗₒ = 0.0077    # Lower frequency
fᵤₚ = 1.0      # Upper frequency
M = 512        # Filter length parameter
α = 7.7        # Window parameter

# create Time vector 
time = range(-M, stop=M, step=1/fₛ)

# Initialize coefficients
ct = zeros(length(time))

# Process DWR-G data
println("Processing DWR_G data now - takes about a minute!\n")
flush(stdout)

# Compute integrals for coefficients
ct = @inbounds [t == 0 ? 0.0 : -0.5 * quadgk(x -> sin(2 * π * x * t) / sin(π * x / fₛ), fₗₒ, fᵤₚ)[1] for t in time]

# Generate Kaiser window
DWR_G_window = kaiser(length(time), α)

p1 = plot(xlabel="time t (s)", ylabel="coefs c", xlims=(-250,250), ylims=(-1,1)) 

# Plot the filter with Kaiser window applied to remove Gibbs phenomenon
p1 = plot!(time, ct, color = :red, lw = 1, label = "Filter")
p1 = plot!(time, ct .* DWR_G_window, color = :blue, label = "Windowed Filter")
p1 = plot!(time, DWR_G_window, color = :lightblue, label = "Kaiser Window")

p2 = plot(time, ct, color = :red, lw = :1, label = "Filter", 
    xlim=(0,25), xticks=(0:5:25), ylim=(-0.7,0), yticks=(-0.7:0.1:0), ylabel="coefs c")
p2 = plot!(time, ct .* DWR_G_window, color=:blue, label="Windowed Filter")

p3 = plot(time, ct, color = :red, lw = :1, label = "Filter",
    xlim=(0,250), xticks=(0:50:250), ylim=(-0.1,0.1), yticks=(-0.1:0.02:0.1),)
p3 = plot!(time, ct .* DWR_G_window, color=:blue, label="Windowed Filter")
p3 = plot!(time, DWR_G_window, color=:lightblue, label="Kaiser Window")

# Define the layout with varying sizes
l = @layout [a{0.5h}; b{0.5w} c{0.5w} [ Plots.grid(1,1) ] ]

# Combine the three plots
title = "Datawell DWR-G filter coefficients"
p1_p2_p3_plot = plot(p1, p2, p3, suptitle = title, leftmargin = 10Plots.mm, layout=l, 
    size=(1000, 800), legendfont=font(10), fg_legend=:false, bg_legend=:transparent, framestyle=:box)

display(p1_p2_p3_plot)

### DWR-G Filter at 2 Hz. and 1.28 Hz. sampling rates

In [ ]:
using Interpolations: interpolate
using Plots: plot, plot!

# Constants and data from previous code
M = 512  # Half of filter length
fₛ = 2.0  # Original sampling frequency

# Time vector for original filter coefficients at 2 Hz
xs = collect(range(-M, stop=M, step=1/fₛ))

# Create the interpolation function (linear interpolation)
interp_linear = interpolate((xs,), windowed_ct, Gridded(Linear()))

# Time vector for the new sampling frequency (1.28 Hz)
xxs = collect(range(-M, stop=M, step=1/1.28))

# Interpolated filter coefficients at 1.28 Hz
yy = [interp_linear(i) for i in xxs]

# Plot the original and resampled filter coefficients
p1 = plot(title="DWR-G Filter", size=(1200,600), ylims=(-0.7,0.7),
          xlabel="time t (s)", ylabel="coefs c", leftmargin=15Plots.mm, bottommargin=15Plots.mm,
          fg_legend=:false, bg_legend=:transparent, framestyle=:box)

# Plot original filter (2 Hz)
plot!(p1, xs, windowed_ct, marker=:x, ms=2, lw=1, label="2 Hz.")

# Plot resampled filter (1.28 Hz)
plot!(p1, xxs, yy, marker=:o, alpha=0.5, ms=2, label="1.28 Hz.")

display(p1)
